<a href="https://colab.research.google.com/github/mariacmartins/bioinformatica-disciplina/blob/main/Aula_11_An%C3%A1lise_de_variantes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de variantes

No processo de identificação de variantes com NGS, por exemplo, o resultado do alinhamento é processado de modo a se identificar posições do genoma onde há **divergência** entre o genoma de interesse e o genoma de referência. 

**Conceitos importantes:**
- **SNPS:** Polimorfismo de nucleotídeo único (em inglês single nucleotide polymorphism) é uma variação na sequência de DNA que afeta somente uma base (adenina, timina, citosina ou guanina) na sequência do genoma entre indivíduos de uma espécie ou entre pares de cromossomos de um individuo. 
- **INDELS:** Indel é um termo da biologia molecular utilizado para classificar pequenas variações genéticas, sendo essas inserções ou deleções de bases no genoma de um organismo. 
- **Arquivo VCF:** O Variant Call Format (VCF) especifica o formato de um arquivo de texto usado em bioinformática para armazenar variações de sequência de genes.


#### Instalação dos pacotes e ferramentas

In [1]:
!apt install sra-toolkit bwa samtools ncbi-entrez-direct bcftools
!pip install snptoolkit

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  blends-common cwltool libauthen-sasl-perl libc-ares2 libdata-dump-perl
  libencode-locale-perl libfile-listing-perl libfont-afm-perl
  libhtml-form-perl libhtml-format-perl libhtml-parser-perl
  libhtml-tagset-perl libhtml-tree-perl libhts2 libhttp-cookies-perl
  libhttp-daemon-perl libhttp-date-perl libhttp-message-perl
  libhttp-negotiate-perl libio-html-perl libio-socket-ssl-perl libjs-bootstrap
  libkdf5-2 liblwp-mediatypes-perl liblwp-protocol-https-perl libmagic-mgc
  libmagic1 libmailtools-perl libmbedcrypto1 libmbedtls10 libncbi-vdb2
  libncbi-wvdb2 libnet-http-perl libnet-smtp-ssl-perl libnet-ssleay-perl
  libtimedate-perl libtry-tiny-perl liburi-perl libwww-perl
  libwww-robotrules-perl med-conf

#### Baixe a sequência e anotação do genoma do vírus SARS-Cov-2 isolado Wuhan-Hu-1 do GenBank (GenBank: NC_045512.2), que servirá como referência. 

In [2]:
!efetch -db nucleotide -id NC_045512.2 -format fasta > NC_045512.2.fasta
!efetch -db nucleotide -id NC_045512.2 -format gbwithparts > NC_045512.2.gb

In [3]:
!head -100 NC_045512.2.fasta

>NC_045512.2 Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome
ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAA
CGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAAC
TAATTACTGTCGTTGACAGGACACGAGTAACTCGTCTATCTTCTGCAGGCTGCTTACGGTTTCGTCCGTG
TTGCAGCCGATCATCAGCACATCTAGGTTTCGTCCGGGTGTGACCGAAAGGTAAGATGGAGAGCCTTGTC
CCTGGTTTCAACGAGAAAACACACGTCCAACTCAGTTTGCCTGTTTTACAGGTTCGCGACGTGCTCGTAC
GTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCAGAGGCACGTCAACATCTTAAAGATGGCACTTGTGG
CTTAGTAGAAGTTGAAAAAGGCGTTTTGCCTCAACTTGAACAGCCCTATGTGTTCATCAAACGTTCGGAT
GCTCGAACTGCACCTCATGGTCATGTTATGGTTGAGCTGGTAGCAGAACTCGAAGGCATTCAGTACGGTC
GTAGTGGTGAGACACTTGGTGTCCTTGTCCCTCATGTGGGCGAAATACCAGTGGCTTACCGCAAGGTTCT
TCTTCGTAAGAACGGTAATAAAGGAGCTGGTGGCCATAGTTACGGCGCCGATCTAAAGTCATTTGACTTA
GGCGACGAGCTTGGCACTGATCCTTATGAAGATTTTCAAGAAAACTGGAACACTAAACATAGCAGTGGTG
TTACCCGTGAACTCATGCGTGAGCTTAACGGAGGGGCATACACTCGCTATGTCGATAACAACTTCTGTGG
CCCTGATGGCTACCCTCTTGAGTGCATTAAAGACCTTCTAGCACGTGCTGG

In [4]:
!head -100 NC_045512.2.gb

LOCUS       NC_045512              29903 bp ss-RNA     linear   VRL 18-JUL-2020
DEFINITION  Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1,
            complete genome.
ACCESSION   NC_045512
VERSION     NC_045512.2
DBLINK      BioProject: PRJNA485481
KEYWORDS    RefSeq.
SOURCE      Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2)
  ORGANISM  Severe acute respiratory syndrome coronavirus 2
            Viruses; Riboviria; Orthornavirae; Pisuviricota; Pisoniviricetes;
            Nidovirales; Cornidovirineae; Coronaviridae; Orthocoronavirinae;
            Betacoronavirus; Sarbecovirus.
REFERENCE   1  (bases 1 to 29903)
  AUTHORS   Wu,F., Zhao,S., Yu,B., Chen,Y.M., Wang,W., Song,Z.G., Hu,Y.,
            Tao,Z.W., Tian,J.H., Pei,Y.Y., Yuan,M.L., Zhang,Y.L., Dai,F.H.,
            Liu,Y., Wang,Q.M., Zheng,J.J., Xu,L., Holmes,E.C. and Zhang,Y.Z.
  TITLE     A new coronavirus associated with human respiratory disease in
            China
  JOURNAL   Nature 579 (7

#### Baixe os dados brutos de sequenciamento de um novo isolado de SARS-Cov-2 obtido na Espanha (SRA: ERR5908782).

In [5]:
!fastq-dump --split-3 ERR5908782

Rejected 48 READS because of filtering out non-biological READS
Read 153205 spots for ERR5908782
Written 153205 spots for ERR5908782


#### Utilizando as ferramentas BWA e Samtools, realize o mapeamento das leituras.

In [6]:
!bwa index -p sarscov2 NC_045512.2.fasta

[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.01 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.00 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index -p sarscov2 NC_045512.2.fasta
[main] Real time: 0.049 sec; CPU: 0.021 sec


In [7]:
!bwa aln sarscov2 ERR5908782_1.fastq > ERR5908782_1.sai
!bwa aln sarscov2 ERR5908782_2.fastq > ERR5908782_2.sai

[bwa_aln] 17bp reads: max_diff = 2
[bwa_aln] 38bp reads: max_diff = 3
[bwa_aln] 64bp reads: max_diff = 4
[bwa_aln] 93bp reads: max_diff = 5
[bwa_aln] 124bp reads: max_diff = 6
[bwa_aln] 157bp reads: max_diff = 7
[bwa_aln] 190bp reads: max_diff = 8
[bwa_aln] 225bp reads: max_diff = 9
[bwa_aln_core] calculate SA coordinate... 4.50 sec
[bwa_aln_core] write to the disk... 0.01 sec
[bwa_aln_core] 153157 sequences have been processed.
[main] Version: 0.7.17-r1188
[main] CMD: bwa aln sarscov2 ERR5908782_1.fastq
[main] Real time: 4.820 sec; CPU: 4.785 sec
[bwa_aln] 17bp reads: max_diff = 2
[bwa_aln] 38bp reads: max_diff = 3
[bwa_aln] 64bp reads: max_diff = 4
[bwa_aln] 93bp reads: max_diff = 5
[bwa_aln] 124bp reads: max_diff = 6
[bwa_aln] 157bp reads: max_diff = 7
[bwa_aln] 190bp reads: max_diff = 8
[bwa_aln] 225bp reads: max_diff = 9
[bwa_aln_core] calculate SA coordinate... 4.47 sec
[bwa_aln_core] write to the disk... 0.02 sec
[bwa_aln_core] 153157 sequences have been processed.
[main] Versio

In [8]:
!bwa sampe sarscov2 ERR5908782_1.sai ERR5908782_2.sai ERR5908782_1.fastq ERR5908782_2.fastq > alignment.sam

[bwa_sai2sam_pe_core] convert to sequence coordinate... 
[infer_isize] (25, 50, 75) percentile: (94, 138, 198)
[infer_isize] low and high boundaries: 200 and 406 for estimating avg and std
[infer_isize] inferred external isize from 36138 pairs: 257.792 +/- 44.163
[infer_isize] skewness: 0.730; kurtosis: -0.318; ap_prior: 1.00e-05
[infer_isize] inferred maximum insert size: 483 (5.11 sigma)
[bwa_sai2sam_pe_core] time elapses: 0.87 sec
[bwa_sai2sam_pe_core] changing coordinates of 44 alignments.
[bwa_sai2sam_pe_core] align unmapped mate...
[bwa_paired_sw] 751 out of 2000 Q17 singletons are mated.
[bwa_paired_sw] 2 out of 112165 Q17 discordant pairs are fixed.
[bwa_sai2sam_pe_core] time elapses: 2.95 sec
[bwa_sai2sam_pe_core] refine gapped alignments... 0.57 sec
[bwa_sai2sam_pe_core] print alignments... 0.66 sec
[bwa_sai2sam_pe_core] 153157 sequences have been processed.
[main] Version: 0.7.17-r1188
[main] CMD: bwa sampe sarscov2 ERR5908782_1.sai ERR5908782_2.sai ERR5908782_1.fastq ERR590

In [9]:
!head -10 alignment.sam

@SQ	SN:NC_045512.2	LN:29903
@PG	ID:bwa	PN:bwa	VN:0.7.17-r1188	CL:bwa sampe sarscov2 ERR5908782_1.sai ERR5908782_2.sai ERR5908782_1.fastq ERR5908782_2.fastq
ERR5908782.1	81	NC_045512.2	13654	37	83M	=	13654	-83	GTTAAGAGACACACTTTCTCTAACTACCAACATGAAGAAACAATTTATAATTTACTTAAGGATTGTCCAGCTGTTGCTAAACA	GGGFGGGGGGGFGGGFCGDGGGGGGGFGGGGGGGDGGFFEGGGGFGGGGGGGGGGGFGF@GGGGGGGGGGGGGGGGGGCCCCC	XT:A:U	NM:i:0	SM:i:37	AM:i:37	X0:i:1	X1:i:0	XM:i:0	XO:i:0	XG:i:0	MD:Z:83
ERR5908782.1	161	NC_045512.2	13654	37	83M	=	13654	83	GTTAAGAGACACACTTTCTCTAACTACCAACATGAAGAAACAATTTATAATTTACTTAAGGATTGTCCAGCTGTTGCTAAACA	CCCCCGGFFEEGGGGGCEFFGGFGGA8CEEF,EAFFGFEFGGGGFEGFGGGGGGGGFGGGF<FGDGFGGGGFEGDGGCGGDEF	XT:A:U	NM:i:0	SM:i:37	AM:i:37	X0:i:1	X1:i:0	XM:i:0	XO:i:0	XG:i:0	MD:Z:83
ERR5908782.2	99	NC_045512.2	23284	60	195M	=	23315	231	TGCTGTCCGTGATCCACAGACACTTGAGATTCTTGACATTACACCATGTTCTTTTGGTGGTGTCAGTGTTATAACACCAGGAACAAATACTTCTAACCAGGTTGCTGTTCTTTATCAGGGTGTTAACTGCACAGAAGTCCCTGTTGCTATTCATGCAGATCAACTTACTCCTACTTGGCGTGTTTATTCTACAGG	GGGGGA

In [10]:
!samtools view -S -b -o alignment.bam alignment.sam

In [11]:
!samtools view -h


Usage: samtools view [options] <in.bam>|<in.sam>|<in.cram> [region ...]

Options:
  -b       output BAM
  -C       output CRAM (requires -T)
  -1       use fast BAM compression (implies -b)
  -u       uncompressed BAM output (implies -b)
  -h       include header in SAM output
  -H       print SAM header only (no alignments)
  -c       print only the count of matching records
  -o FILE  output file name [stdout]
  -U FILE  output reads not selected by filters to FILE [null]
  -t FILE  FILE listing reference names and lengths (see long help) [null]
  -L FILE  only include reads overlapping this BED FILE [null]
  -r STR   only include reads in read group STR [null]
  -R FILE  only include reads with read group listed in FILE [null]
  -q INT   only include reads with mapping quality >= INT [0]
  -l STR   only include reads in library STR [null]
  -m INT   only include reads with number of CIGAR operations consuming
           query sequence >= INT [0]
  -f INT   only include reads with a

In [12]:
!samtools sort -o alignment.sorted.bam alignment.bam

#### Utilize as ferramentas Bcftools e Vcfutils para identificar as variantes genéticas a partir do arquivo BAM (ordenado).

In [13]:
!bcftools mpileup -f NC_045512.2.fasta -O z -o alignment.vcf.gz -m 10 alignment.sorted.bam

[mpileup] 1 samples in 1 input files


In [14]:
!bcftools call -c -v --ploidy 1 alignment.vcf.gz > variants.raw.vcf

In [15]:
!head -50 variants.raw.vcf

##fileformat=VCFv4.2
##FILTER=<ID=PASS,Description="All filters passed">
##bcftoolsVersion=1.7+htslib-1.7-2
##bcftoolsCommand=mpileup -f NC_045512.2.fasta -O z -o alignment.vcf.gz -m 10 alignment.sorted.bam
##reference=file://NC_045512.2.fasta
##contig=<ID=NC_045512.2,length=29903>
##ALT=<ID=*,Description="Represents allele(s) other than observed.">
##INFO=<ID=INDEL,Number=0,Type=Flag,Description="Indicates that the variant is an INDEL.">
##INFO=<ID=IDV,Number=1,Type=Integer,Description="Maximum number of reads supporting an indel">
##INFO=<ID=IMF,Number=1,Type=Float,Description="Maximum fraction of reads supporting an indel">
##INFO=<ID=DP,Number=1,Type=Integer,Description="Raw read depth">
##INFO=<ID=VDB,Number=1,Type=Float,Description="Variant Distance Bias for filtering splice-site artefacts in RNA-seq data (bigger is better)",Version="3">
##INFO=<ID=RPB,Number=1,Type=Float,Description="Mann-Whitney U test of Read Position Bias (bigger is better)">
##INFO=<ID=MQB,Number=1,Type=Floa

In [16]:
!vcfutils.pl varFilter -d 30 variants.raw.vcf > variants.filtered.vcf

In [17]:
!head -50 variants.filtered.vcf

##fileformat=VCFv4.2
##FILTER=<ID=PASS,Description="All filters passed">
##bcftoolsVersion=1.7+htslib-1.7-2
##bcftoolsCommand=mpileup -f NC_045512.2.fasta -O z -o alignment.vcf.gz -m 10 alignment.sorted.bam
##reference=file://NC_045512.2.fasta
##contig=<ID=NC_045512.2,length=29903>
##ALT=<ID=*,Description="Represents allele(s) other than observed.">
##INFO=<ID=INDEL,Number=0,Type=Flag,Description="Indicates that the variant is an INDEL.">
##INFO=<ID=IDV,Number=1,Type=Integer,Description="Maximum number of reads supporting an indel">
##INFO=<ID=IMF,Number=1,Type=Float,Description="Maximum fraction of reads supporting an indel">
##INFO=<ID=DP,Number=1,Type=Integer,Description="Raw read depth">
##INFO=<ID=VDB,Number=1,Type=Float,Description="Variant Distance Bias for filtering splice-site artefacts in RNA-seq data (bigger is better)",Version="3">
##INFO=<ID=RPB,Number=1,Type=Float,Description="Mann-Whitney U test of Read Position Bias (bigger is better)">
##INFO=<ID=MQB,Number=1,Type=Floa

#### Utilize a ferramenta Snptoolkit para anotar as variantes com base na anotação do genoma.

In [18]:
!snptoolkit annotate -i variants.filtered.vcf -g NC_045512.2.gb

[12:55:42] [INFO] [1 CPUs requested out of 2 detected on this machine]
[12:55:42] [INFO] [snpToolkit is filtering and annotating your SNPs]
  0% 0/1 [00:00<?, ?it/s][12:55:42] [INFO] [snpToolkit output files for SNPs data will be located in folders snpToolkit_SNPs_output_2021-05-24_at_12.55.42]
[12:55:42] [INFO] [snpToolkit output files for INDELs data will be located in folders snpToolkit_INDELS_output_2021-05-24_at_12.55.42]
100% 1/1 [00:00<00:00,  7.41it/s]


#### Analisando os dados de SNPs

In [19]:
import pandas as pd
snps_df = pd.read_excel('snps_sarscov2.xlsx')
snps_df.head()

,##Coordinates,REF,SNP,Depth,Nb of reads REF,Nb reads SNPs,Ratio,Quality,Annotation,Product,Orientation,Coordinates in gene,Ref codon,SNP codon,Ref AA,SNP AA,Coordinates protein,Effect,Location
0,241,C,T,203,0,103,1.00,225007,intergenic,.,+,25 bp from GU280_gp01|ORF1ab,.,.,.,.,.,.,NC_045512.2: Severe acute respiratory syndrome...
1,913,C,T,91,0,81,1.00,225007,GU280_gp01|ORF1ab,ORF1a polyprotein,+,648,TCC,TC[T],S,S,216,Syn,NC_045512.2: Severe acute respiratory syndrome...
2,2698,T,C,299,2,198,0.99,225007,GU280_gp01|ORF1ab,ORF1a polyprotein,+,2433,AAT,AA[C],N,N,811,Syn,NC_045512.2: Severe acute respiratory syndrome...
3,3037,C,T,302,0,219,1.00,225007,GU280_gp01|ORF1ab,ORF1a polyprotein,+,2772,TTC,TT[T],F,F,924,Syn,NC_045512.2: Severe acute respiratory syndrome...
4,3267,C,T,300,0,256,1.00,225007,GU280_gp01|ORF1ab,ORF1a polyprotein,+,3002,ACT,A[T]T,T,I,1001,NS,NC_045512.2: Severe acute respiratory syndrome...
